In [124]:
%%capture
!pip install readability-lxml

In [125]:
import os 
import bs4
import json
import nltk
import time 
import codecs
import pickle 
import logging
import random,math



from nltk.tokenize import WordPunctTokenizer
from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader
from nltk.corpus import LazyCorpusLoader, CategorizedPlaintextCorpusReader,CategorizedCorpusReader



from readability.readability import Unparseable
from readability.readability import Document as Paper
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize

log = logging.getLogger("readability.readability")
log.setLevel('WARNING')
# from readability.readability import Document as Paper
# Module Variables
#https://github.com/DistrictDataLabs/PyCon2016/tree/master/notebooks/tutorial

In [126]:
ROOT = os.getcwd() 
nomi = os.path.join(ROOT, "nomi", "frankie")
pickle_dir = os.path.join(ROOT, "nomi", 'pickled')
pickle_dir
# CORPUS   = os.path.join(ROOT, "fixtures", "corpus")
# GALACTIC = os.path.join(ROOT, "fixtures", "galactic")
# TAGGED   = os.path.join(ROOT, "fixtures", "preprocessed")

'/Users/ankush.singal/Desktop/atap/snippets/ch04/nomi/pickled'

In [127]:
#https://bitcoden.com/answers/creating-a-custom-categorized-corpus-in-nltk-and-python
#https://python.hotexamples.com/examples/nltk.corpus/PlaintextCorpusReader/sents/python-plaintextcorpusreader-sents-method-examples.html
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'(\w+)/*'
corpus = CategorizedPlaintextCorpusReader(nomi, r'.*\.txt', cat_pattern=CAT_PATTERN)


In [128]:
#https://www.districtdatalabs.com/named-entity-recognition-and-classification-for-entity-extraction
def get_documents(path=nomi):
    """
    Returns a filtered list of paths to PDF files representing our corpus.
    """
    for name in os.listdir(path):
        if name.startswith('c') and name.endswith('.txt'):
            yield os.path.join(path, name)

# Print the total number of documents
print(len(list(get_documents())))

2


In [129]:
for prop in dir(corpus):
    if not prop.startswith('_'): print(prop)

CorpusView
abspath
abspaths
categories
citation
encoding
ensure_loaded
fileids
license
open
paras
raw
readme
root
sents
words


In [130]:
corpus.categories()

['corpus', 'gutenberg']

In [131]:
# Print out the fileids
for fid in corpus.fileids(): print(fid)

corpus-original.txt
corpus.txt
gutenberg.txt


In [132]:
words = nltk.FreqDist(corpus.words())
count = sum(words.values())
vocab = len(words)

print("Corpus contains a vocabulary of {} and a word count of {}.".format(
    count, vocab
))

Corpus contains a vocabulary of 6178253 and a word count of 79228.


#USING REGULAR EXPRESSIONS TO EXTRACT SPECIFIC SECTIONS
#To begin our exploration of regular expressions (a.k.a. "regex"), it's important to point out some good resources for those new to the topic. An excellent resource may be found in Johns Hopkins University's Coursera video titled Getting and Cleaning Data.

In [133]:
import re
def titles(fileid=None,corpus=corpus):
    """
    Use a regular expression to extract the titles from the corpus.
    """
    pattern = re.compile(r'^(.*)[\s]+[\s]?(.*)?')

    if fileid is not None:
        match = pattern.search(corpus.raw(fileid))
        yield " ".join(map(lambda s: s.strip(), match.groups()))
    else:
        for fileid in corpus.fileids():
            # Search for a pattern match
            match = pattern.search(corpus.raw(fileid))

            if match:
                # If we find one, yield the space joined groups.
                yield " ".join(map(lambda s: s.strip(), match.groups()))

for idx, title in enumerate(titles()):
        print(title)
        if idx >= 10:
             break

PRIDE AND PREJUDICE By Jane Austen
However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. "My dear Mr.
PRIDE AND PREJUDICE By Jane Austen


In [134]:
# View the first 12 sentences in Star Trek
# for idx, sent in enumerate(corpus.sents(fileids='corpus-original.txt')):
#     print(sent)
#     if idx > 12: break

In [135]:
# Corpus Patterns 
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'(\w+)/*'
PKL_PATTERN = r'.+\.pickle'     # Pickled files end in .pickle


class BaleenCorpusReader(CategorizedCorpusReader, CorpusReader):
    """
    A corpus reader for the raw JSON Baleen documents that have not been
    preprocessed and have the complete feed information exported from Mongo.
    """

    # Tags to extract as paragraphs from the HTML text
    TAGS = [
        'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li'
    ]

    def __init__(self, root, fileids=DOC_PATTERN, tags=None,
                 word_tokenizer=WordPunctTokenizer(),
                 sent_tokenizer=nltk.data.LazyLoader(
                    'tokenizers/punkt/english.pickle'),
                 encoding='utf8', **kwargs):
        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining arguments
        are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids, encoding)

        self._word_tokenizer = word_tokenizer
        self._sent_tokenizer = sent_tokenizer
        self._good_tags = tags or self.TAGS

#     def feeds(self):
#         """
#         Opens and returns the collection of feeds associated with the corpus.
#         """
#         data = self.open('feeds.json')
#         return json.load(data)

    def resolve(self, fileids, categories):
            """
            Returns a list of fileids or categories depending on what is passed
            to each internal corpus reader function. Implemented similarly to
            the NLTK ``CategorizedPlaintextCorpusReader``.
            """
            if fileids is not None and categories is not None:
                raise ValueError("Specify fileids or categories, not both")

            if categories is not None:
                return self.fileids(categories)
            return fileids

    def docs(self, fileids=None, categories=None):
        """
        Returns the complete text of a document, closing the document
        after we are done reading it and yielding it in a memory safe fashion.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, encoding in self.abspaths(fileids, include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as f:
                yield f.read()
    
    def sizes(self, fileids=None, categories=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            yield os.path.getsize(path)            

    def fields(self, fields, fileids=None, categories=None):
        """
        Helper function to extract particular fields from the json documents.
        Fields can be a string or an iterable of fields. If just one field is
        passed in, then the values are returned, otherwise dictionaries of
        the requsted fields are returned.

        This method doesn't raise KeyErrors nor does it yield None values if
        the document doesn't contain a particular field.

        For example to get title and pubdate from the documents:

            corpus.fields(['title', 'pubdate'])

        Or to simply get all of the summaries:

            corpus.fields('summary')

        Note: there is not yet support for nested fields.
        """
        if isinstance(fields, string_types):
            fields = [fields,]

        if len(fields) == 1:
            for doc in self.docs(fileids, categories):
                if fields[0] in doc:
                    yield doc[fields[0]]

        else:
            for doc in self.docs(fileids, categories):
                yield {
                    key: doc.get(key, None)
                    for key in fields
                }

#     def html(self, fileids=None, categories=None, readability=True):
#         """
#         Returns the HTML content from each JSON document for every file in
#         the corpus, ensuring that it exists. Note, this simply returns the
#         HTML strings, it doesn't perform any parsing of the HTML.

#         If readability is True, clean HTML is returned.
#         """
#         ## Returns a generator of documents.
#         html = self.fields('content', fileids, categories)
#         if readability:
#             for doc in html:
#                 try:
#                     yield Paper(doc).summary()
#                 except:
#                     continue
#         else:
#             for doc in html:
#                 yield doc
                
    def paras(self, fileids=None, categories=None):
        """
        Uses splitlines() to parse the paragraphs from plain text.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)
        
        for doc in self.docs(fileids):
            for par in doc.splitlines():
                if len(par) > 0:
                    yield par
    def sents(self, fileids=None, categories=None):
        """
        Uses the built in sentence tokenizer to extract sentences from the
        paragraphs. Note that this method uses BeautifulSoup to parse HTML.
        """
        for paragraph in self.paras(fileids, categories):
            for sentence in self._sent_tokenizer.tokenize(paragraph):
                yield sentence

    def words(self, fileids=None, categories=None):
        """
        Uses the built in word tokenizer to extract tokens from sentences.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)
        
        for sentence in self.sents(fileids):
            for token in wordpunct_tokenize(sentence):
                yield token

                
    def tokenize(self, fileids=None, categories=None):
        """
        Segments, tokenizes, and tags a document in the corpus.
        """
        for paragraph in self.paras(fileids, categories):
            yield [
                pos_tag(wordpunct_tokenize(sent))
                for sent in sent_tokenize(paragraph)
            ]

    def describe(self, fileids=None, categories=None):
        """
        Performs a single pass of the corpus and
        returns a dictionary with a variety of metrics
        concerning the state of the corpus.
        """
        started = time.time()

        # Structures to perform counting.
        counts  = nltk.FreqDist()
        tokens  = nltk.FreqDist()

        # Perform single pass over paragraphs, tokenize and count
        for para in self.paras(fileids, categories):
            counts['paras'] += 1

            for sent in sent_tokenize(para):
                counts['sents'] += 1

                for word in wordpunct_tokenize(sent):
                    counts['words'] += 1
                    tokens[word] += 1

        # Compute the number of files and categories in the corpus
        n_fileids = len(self.resolve(fileids, categories) or self.fileids())
        n_topics  = len(self.categories(self.resolve(fileids, categories)))

        # Return data structure with information
        return {
            'files':  n_fileids,
            'categories': n_topics,
            'paragraphs':  counts['paras'],
            'sentences':  counts['sents'],
            'words':  counts['words'],
            'vocabulary_size':  len(tokens),
            'lexical_diversity': float(counts['words']) / float(len(tokens)),
            'paras_per_doc':  float(counts['paras']) / float(n_fileids),
            'words_per_doc':  float(counts['words']) / float(n_fileids),
            'sents_per_para':  float(counts['sents']) / float(counts['paras']),
            'secs':   time.time() - started,
        }
    

In [136]:
# Load the Baleen Corpus Reader
corpus2 = BaleenCorpusReader(nomi, r'.+\.txt')
print(
    "Baleen corpus contains {} files in {} categories".format(
        len(corpus.fileids()), len(corpus.categories())
    )
)

Baleen corpus contains 3 files in 2 categories


In [137]:
print("{} documents in {} categories".format(len(corpus2.fileids()), len(corpus2.categories())))

3 documents in 2 categories


In [138]:
# Optional to run
# Print statistics about each category. 
words = nltk.ConditionalFreqDist([
        (category, word) 
        for category in corpus2.categories()
        for word in corpus2.words(categories=category)
    ])

for category, dist in words.items():
    wc = sum(dist.values())
    vb = len(dist) 
    print("{} has {:>,} vocab and {:>,} words".format(category, vb, wc))

corpus has 79,175 vocab and 5,086,415 words
gutenberg has 32,970 vocab and 1,092,764 words


In [139]:
#https://github.com/wilkens-teaching/textmining/blob/master/exercises/03%20corpus%20reader%20advanced.ipynb
description = corpus2.describe()
for key in description:
    print(key, description[key])

files 3
categories 2
paragraphs 272037
sentences 290805
words 6179179
vocabulary_size 79175
lexical_diversity 78.04457215029997
paras_per_doc 90679.0
words_per_doc 2059726.3333333333
sents_per_para 1.0689906152471906
secs 11.139745950698853


In [140]:
class PickledCorpusReader(CategorizedCorpusReader, CorpusReader):

    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining arguments
        are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids)

    def resolve(self, fileids, categories):
            """
            Returns a list of fileids or categories depending on what is passed
            to each internal corpus reader function. Implemented similarly to
            the NLTK ``CategorizedPlaintextCorpusReader``.
            """
            if fileids is not None and categories is not None:
                raise ValueError("Specify fileids or categories, not both")

            if categories is not None:
                return self.fileids(categories)
            return fileids

    def docs(self, fileids=None, categories=None):
        """
        Returns the complete text of a document, closing the document
        after we are done reading it and yielding it in a memory safe fashion.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, encoding in self.abspaths(fileids, include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as f:
                yield f.read()
    

    def paras(self, fileids=None, categories=None):
        """
        Uses splitlines() to parse the paragraphs from plain text.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)
        
        for doc in self.docs(fileids):
            for par in doc.splitlines():
                if len(par) > 0:
                    yield par
    def sents(self, fileids=None, categories=None):
        """
        Uses the built in sentence tokenizer to extract sentences from the
        paragraphs.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)
        
        for paragraph in self.paras(fileids):
            for sentence in sent_tokenize(paragraph):
                yield sentence


    def tagged(self, fileids=None, categories=None):
        for sent in self.sents(fileids, categories):
            for token in sent:
                yield token

    def words(self, fileids=None, categories=None):
        """
        Uses the built in word tokenizer to extract tokens from sentences.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)
        
        for sentence in self.sents(fileids):
            for token in wordpunct_tokenize(sentence):
                yield token


if __name__ == '__main__':
    from collections import Counter

    corpus3 = PickledCorpusReader(nomi,r'.+\.txt')
    words  = Counter(corpus3.words())

    print("{:,} vocabulary {:,} word count".format(len(words.keys()), sum(words.values())))
#https://github.com/wilkens-teaching/textmining/blob/master/exercises/03%20corpus%20reader%20advanced.ipynb
#https://github.com/DistrictDataLabs/PyCon2016/blob/master/notebooks/tutorial/Machine%20Learning.ipynb

79,175 vocabulary 6,179,179 word count


"""Our next step will be to prepare our preprocessed text data for machine learning by encoding it as vectors. We’ll weigh several techniques for vector encoding, and discuss how to wrap that encoding process in a pipeline to allow for systematic loading, normalization, and feature extraction."""

In [141]:
#USING BaseEstimator and Transformermin
#https://bbengfort.github.io/2016/05/text-classification-nltk-sckit-learn/
#https://www.kaggle.com/code/baghern/a-deep-dive-into-sklearn-pipelines
#https://towardsdatascience.com/customizing-sklearn-pipelines-transformermixin-a54341d8d624
#https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-ef792bbb3260
#https://stackoverflow.com/questions/70550018/sklearn-custom-transformers-with-pipeline-all-the-input-array-dimensions-for-th

In [152]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as tts

class CorpusLoader(object):
    """
    The corpus loader knows how to deal with an NLTK corpus at the top of a
    pipeline by simply taking as input a corpus to read from. It exposes both
    the data and the labels and can be set up to do cross-validation.
    If a number of folds is passed in for cross-validation, then the loader
    is smart about how to access data for train/test splits. Otherwise it will
    simply yield all documents in the corpus.
    """

    def __init__(self, corpus, folds=12, shuffle=True):
        self.n_docs = len(corpus.fileids())
        self.corpus = corpus
        self.folds  = KFold(n_splits=folds, shuffle=shuffle)
        self.shuffle = True

        if folds is not None:
            # Generate the KFold cross validation for the loader.
            self.folds = KFold(self.n_docs, folds, shuffle)

    @property
    def n_folds(self):
        """
        Returns the number of folds if it exists; 0 otherwise.
        """
        if self.folds is None: return 0
        return self.folds.n_folds

    def fileids(self, fold=None, train=False, test=False):
        """
        Returns a listing of the documents filtering to retreive specific
        data from the folds/splits. If no fold, train, or test is specified
        then the method will return all fileids.
        If a fold is specified (should be an integer between 0 and folds),
        then the loader will return documents from that fold. Further, train
        or test must be specified to split the fold correctly.
        """
        if fold is None:
            # If no fold is specified, return all the fileids.
            return self.corpus.fileids()

        # Otherwise, identify the fold specifically and get the train/test idx
        for fold_idx, (train_idx, test_idx) in enumerate(self.folds):
            if fold_idx == fold: break
        else:
            # We have discovered no correct fold.
            raise ValueError(
                "{} is not a fold, specify an integer less than {}".format(
                    fold, self.folds.n_folds
                )
            )

        # Now determine if we're in train or test mode.
        if not (test or train) or (test and train):
            raise ValueError(
                "Please specify either train or test flag"
            )

        # Select only the indices to filter upon.
        indices = train_idx if train else test_idx
        return [
            fileid for doc_idx, fileid in enumerate(self.corpus.fileids())
            if doc_idx in indices
        ]

    def labels(self, fold=None, train=False, test=False):
        """
        Fit will load a list of the labels from the corpus categories.
        If a fold is specified (should be an integer between 0 and folds),
        then the loader will return documents from that fold. Further, train
        or test must be specified to split the fold correctly.
        """
        return [
            self.corpus.categories(fileids=fileid)[0]
            for fileid in self.fileids(fold, train, test)
        ]

    def documents(self, fold=None, train=False, test=False):
        """
        A generator of documents being streamed from disk. Each document is
        a list of paragraphs, which are a list of sentences, which in turn is
        a list of tuples of (token, tag) pairs. All preprocessing is done by
        NLTK and the CorpusReader object this object wraps.
        If a fold is specified (should be an integer between 0 and folds),
        then the loader will return documents from that fold. Further, train
        or test must be specified to split the fold correctly. This method
        allows us to maintain the generator properties of document reads.
        """
        for fileid in self.fileids(fold, train, test):
            yield list(self.corpus.tagged(fileids=fileid))



if __name__ == '__main__':
    from reader import PickledCorpusReader

    corpus4 = PickledCorpusReader(nomi,r'.+\.txt')
    loader = CorpusLoader(corpus, folds=12)

    for fid in loader.fileids(0):
        print(fid)

TypeError: __init__() takes from 1 to 2 positional arguments but 4 were given

In [121]:
%%capture
!pip install ipyvizzu

In [122]:
import pandas as pd
from ipyvizzu import Chart, Data, Config

data_frame = pd.read_csv(
    "https://raw.githubusercontent.com/vizzuhq/ipyvizzu/main/docs/examples/stories/titanic/titanic.csv"
)
data = Data()
data.add_data_frame(data_frame)

chart = Chart(width="640px", height="360px")

chart.animate(data)

chart.animate(
    Config(
        {
            "x": "Count",
            "y": "Sex",
            "label": "Count",
            "title": "Passengers of the Titanic",
        }
    )
)
chart.animate(
    Config(
        {
            "x": ["Count", "Survived"],
            "label": ["Count", "Survived"],
            "color": "Survived",
        }
    )
)
chart.animate(Config({"x": "Count", "y": ["Sex", "Survived"]}))